In [34]:
# import module for retrieving info from webpage
import urllib3

# contact python
http = urllib3.PoolManager()

# grab data for top 2019 movies from metacritic.com
url = 'https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2019&sort=desc&view=detailed'

# request info from meta critic by spoofing as browser
# credit to Sav Bell information on how to get around meta critic security 
resp = http.request('GET', url, headers={'User-Agent': 'Mozilla/5.0'})

# decode raw data to usable information 
webtext = resp.data.decode('utf-8')

# check success 
print(f"Fetched {len(webtext)} characters from {url} (status: {resp.status})")

Fetched 512485 characters from https://www.metacritic.com/browse/movies/score/metascore/year/filtered?year_selected=2019&sort=desc&view=detailed (status: 200)


In [35]:
# import module for regular expressions 
import re

# use regular expressions to search for titles, dates, descriptions, metascores, and thumbnails 
# credit to Alex Song for sharing how they fixed issues with regular expressions being read in python 
titles = re.findall("class=\"title\"><h3>(.*)<\/h3>", webtext)
dates = re.findall("class=\"clamp-details\">\n\s+<span>(.*)<\/", webtext)
descriptions = re.findall("class=\"summary\">\n\s+(.*)", webtext)
metascores = re.findall("class=\"metascore_w large movie positive\">(.*)<", webtext)
# credit to Braydon Gemar for the thumbnail portion of the regular expressions. I had trouble with the one I produced working.
thumbnails = re.findall('<a href="\S+ src="(.*)" alt="(?!Metacritic logo")', webtext)

In [36]:
# show retrieved info nicely within an array
# create empty list for array
movies = []

# append movie list to show all needed information
# delved into my old PROG 108 replit files to find correct format for append and counter
for i in range(0,100): 
    movies.append(str("Movie Title: " + titles[i] + "\n" + 
                      "Release Date: " + dates[i] + "\n" + 
                      "Description: " + descriptions[i] + "\n" + 
                      "Metascore: " + metascores[i] + "\n"
                      "Thumbnail: " + thumbnails[i] + "\n"))

In [37]:
# set counter to start
i = 0
# set up loop to function while there are movies to show
while i <len(movies): 
    print(movies[i] + "\n") 
    # set counter to increase after each movie returned
    i = i + 1

Movie Title: Parasite
Release Date: October 11, 2019
Description: Meet the Park Family: the picture of aspirational wealth. And the Kim Family, rich in street smarts but not much else. Be it chance or fate, these two houses are brought together and the Kims sense a golden opportunity. Masterminded by college-aged Ki-woo, the Kim children expediently install themselves as tutor and art therapist, to the Parks. Soon, a symbiotic relationship forms between the two families. The Kims provide “indispensable” luxury services while the Parks obliviously bankroll their entire household. When a parasitic interloper threatens the Kims’ newfound comfort, a savage, underhanded battle for dominance breaks out, threatening to destroy the fragile ecosystem between the Kims and the Parks. [Neon]
Metascore: 96
Thumbnail: https://static.metacritic.com/images/products/movies/5/b0eb57fd63df90ca7e654c788f5851a8-98.jpg


Movie Title: Portrait of a Lady on Fire
Release Date: December 6, 2019
Description: Fra

In [38]:
# grab credentials for SQL login from a json file to not leak data
import json

with open('/Users/vanes/ThisPC/Documents/GitHub/credentials.json') as f:
    data = json.load(f)
    username = data['username'] 
    password = data['password'] 

In [39]:
# connect jupyter notebook to SQL server
import pyodbc

conn2 = pyodbc.connect("Driver={SQL Server};"
                       "Server=FALCON-HQ\SQLEXPRESS;"
                       "Database=DA320;"
                       "Trusted_Connection=yes;"
                       "UID" + username + ";"
                       "PWD" + password)

In [40]:
# insert scraped data into SQL database
cursor = conn2.cursor()
SQL = "INSERT INTO Reviews (MovieTitle, ReleaseDate, Description, Metascore) values (?,?,?,?)"
Arguments = "Movie Title", "Release Date", "Description", "Metascore" 
i = 0
while i < len(titles) and i < len(dates) and i < len(descriptions) and i*2 < len(metascores):
              Arguments = titles[i], dates[i], descriptions[i], metascores[i*2]
              cursor.execute(SQL, Arguments)
              cursor.commit()
              i = i + 1